In [1]:
# Libraries to help with reading and manipulating data
import numpy as np
import pandas as pd
import pickle
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction import DictVectorizer

# for regression models
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

# To help with data visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

#performance
from sklearn.metrics import mean_squared_error

# To supress warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("mlops-project")

2023/07/24 08:20:00 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/07/24 08:20:00 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade 3500859a5d39 -> 7f2a7d5fae7d, add datasets inputs input_tags tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/home/tapji/.conda/mlops-project/mlruns/1', creation_time=1689852646451, experiment_id='1', last_update_time=1689852646451, lifecycle_stage='active', name='mlops-project', tags={}>

### Importing dataset

In [3]:
def read_dataframe(filename: str):
    df = pd.read_csv(filename, encoding='iso-8859-1')
    
    # Remove '£' and ',' sign from 'Total Cost/ 10000 miles' column and convert to numeric values
    df['Annual fuel Cost 10000 Miles'] = df['Annual fuel Cost 10000 Miles'].str.replace('£', '')
    df['Annual fuel Cost 10000 Miles'] = df['Annual fuel Cost 10000 Miles'].str.replace(',', '').astype(int)
    df['Annual Electricity cost / 10000 miles'] = df['Annual Electricity cost / 10000 miles'].str.replace('£', '')
    df['Annual Electricity cost / 10000 miles'] = df['Annual Electricity cost / 10000 miles'].str.replace(',', '').astype(int)
    df['Total cost / 10000 miles'] = df['Total cost / 10000 miles'].str.replace('£', '')
    df['Total cost / 10000 miles'] = df['Total cost / 10000 miles'].str.replace(',', '').astype(int)

    # dropping irrelevant features
    df.drop(['Manufacturer', 'Model', 'Description','Transmission', 'Engine Power (Kw)', 'Engine Power (PS)',
      'Electric energy consumption Miles/kWh', 'wh/km', 'Diesel VED Supplement', 'Testing Scheme', 'Euro Standard', 'Maximum range (Miles)',
      'WLTP Imperial Low', 'WLTP Imperial Medium', 'WLTP Imperial High','WLTP Imperial Extra High', 'WLTP Imperial Combined',
      'WLTP Imperial Combined (Weighted)', 'WLTP Metric Low','WLTP Metric Medium', 'WLTP Metric High', 'WLTP Metric Extra High',
      'WLTP Metric Combined', 'WLTP Metric Combined (Weighted)','WLTP CO2 Weighted', 'Equivalent All Electric Range Miles', 'Equivalent All Electric Range KM',
      'THC Emissions [mg/km]', 'Electric Range City Miles', 'RDE NOx Urban', 'Powertrain', 'Annual fuel Cost 10000 Miles', 'Electric Range City Km', 'Noise Level dB(A)',
      'RDE NOx Combined', 'Emissions CO [mg/km]', 'Emissions NOx [mg/km]', 'THC + NOx Emissions [mg/km]', 'Annual Electricity cost / 10000 miles', 'Maximum range (Km)'], axis=1, inplace=True)

    # inputting missing values
    columns_to_impute = ['WLTP CO2', 'Particulates [No.] [mg/km]']
    
    imputer = SimpleImputer(strategy='mean')
    df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])

   # Converting object data type to categorical

    for feature in df.columns: 
      if df[feature].dtype == 'object': 
         df[feature] = pd.Categorical(df[feature])# Replace strings with an integer

    # Renaming columns
    
    df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

    return df


In [4]:
df = read_dataframe('./data/emission_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4625 entries, 0 to 4624
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   Engine_Capacity             4625 non-null   int64   
 1   Fuel_Type                   4625 non-null   category
 2   WLTP_CO2                    4625 non-null   float64 
 3   Total_cost_/_10000_miles    4625 non-null   int64   
 4   Particulates_[No.]_[mg/km]  4625 non-null   float64 
dtypes: category(1), float64(2), int64(2)
memory usage: 149.5 KB


In [7]:
def train_test(df):
    # independent variables
    X = df.drop(["WLTP_CO2"], axis=1)
    # dependent variable 
    y = df["WLTP_CO2"]

    # Adding intercept to the dataset
    X = pd.get_dummies(X, drop_first=True)   
    X = sm.add_constant(X)
    
    # Splitting X and y into train and test sets in a 70:30 ratio
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.30, random_state=1
    )
    return X_train, X_test, y_train, y_test

In [8]:
X_train, X_test, y_train, y_test = train_test(df)

In [16]:
# Training the model

def ols_regression(X_train, X_test, y_train, y_test): 
    mlflow.end_run()
    with mlflow.start_run():
        mlflow.set_tag("model", "ols_regression")
        olsmod = sm.OLS(y_train, X_train)
        olsres = olsmod.fit()
    
        # Identify columns in the training and test dataset
        train_columns = set(X_train.columns)
        test_columns = set(X_test.columns)
    
        # Compare columns training and testing dataset columns
        columns_to_drop = test_columns - train_columns
    
        # Drop columns from the test dataset
        X_test.drop(columns=columns_to_drop, inplace=True)

        # Making predictions on the test set
        y_pred = olsres.predict(X_test)
        rmse1 = np.sqrt(mean_squared_error(y_train, olsres.fittedvalues))
        rmse2 = np.sqrt(mean_squared_error(y_test, y_pred))
        mlflow.log_metric("rmse", rmse2)
    # return the regression summary as a string
        with open("models/preprocessor.b", "wb") as f_out:
            pickle.dump(olsres, f_out)
            mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
            mlflow.sklearn.log_model(olsres, "OLS_Model")
    return rmse1, rmse2, olsres

In [17]:
rmse1, rmse2, olres = ols_regression(X_train, X_test, y_train, y_test)

In [11]:
print(rmse1, rmse2)

17.613735102497987 13.022063151691114


In [10]:
print(olres.summary())

                            OLS Regression Results                            
Dep. Variable:               WLTP_CO2   R-squared:                       0.919
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     3657.
Date:                Fri, 21 Jul 2023   Prob (F-statistic):               0.00
Time:                        10:51:32   Log-Likelihood:                -13879.
No. Observations:                3237   AIC:                         2.778e+04
Df Residuals:                    3226   BIC:                         2.785e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const       

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4625 entries, 0 to 4624
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   Engine_Capacity             4625 non-null   int64   
 1   Fuel_Type                   4625 non-null   category
 2   WLTP_CO2                    4625 non-null   float64 
 3   Total_cost_/_10000_miles    4625 non-null   int64   
 4   Particulates_[No.]_[mg/km]  4625 non-null   float64 
dtypes: category(1), float64(2), int64(2)
memory usage: 149.5 KB


In [12]:
def lasso_prep(df):
    
    # Splitting X and y into train and test sets in a 70:30 ratio
     # independent variables
    X = df.drop(["WLTP_CO2"], axis=1)
    # dependent variable 
    y = df["WLTP_CO2"]
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.30, random_state=1)
    
    categorical = ['Fuel_Type']
    numerical = ['Engine_Capacity', 'Total_cost_/_10000_miles', 'Particulates_[No.]_[mg/km]']
    dv = DictVectorizer()
    train_dicts = X_train[categorical + numerical].to_dict(orient='records')
    X_train = dv.fit_transform(train_dicts)
    test_dicts = X_test[categorical + numerical].to_dict(orient='records')
    X_test = dv.transform(test_dicts)

    return X_train, X_test, y_train, y_test

In [57]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((olres), f_out)

In [13]:
X_train, X_test, y_train, y_test = lasso_prep(df)

In [60]:

def lasso_regression(X_train, X_test, y_train, y_test):
    
    mlflow.end_run()
    with mlflow.start_run():
    
        mlflow.set_tag('Data Scientist', 'Tapji')
        mlflow.log_param('train-data-path', './data/emission_data.csv')
        alpha = 0.0001
        mlflow.log_param('alpha', alpha)

    
        target = 'WLTP_CO2'
        y_train = y_train.values.ravel()  # Convert target to a 1-dimensional array
        y_test = y_test.values.ravel()    # Convert target to a 1-dimensional array

        lr = Lasso(alpha)  # Assuming you want to use a specific alpha value

        # Fit the Lasso model to the training data
        lr.fit(X_train, y_train)

        # Making predictions on the test set
        y_pred = lr.predict(X_test)

        # Calculate RMSE for training set
        rmse3 = np.sqrt(mean_squared_error(y_train, lr.predict(X_train)))

        # Calculate RMSE for test set
        rmse4 = np.sqrt(mean_squared_error(y_test, y_pred))
        mlflow.log_metric('rmse_training', rmse3)
        mlflow.log_metric('rmse_test', rmse4)
        
        mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle/lin_reg.bin")
    return rmse3, rmse4, lr    
        

In [61]:
rmse3, rmse4, lr = lasso_regression(X_train, X_test, y_train, y_test)

In [34]:
print(rmse3, rmse4)

17.613735319974264 13.022177378370465


#### Using XGBoost for Training

In [42]:
# XGBoost
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [43]:
train = xgb.DMatrix(X_train, label=y_train)
test = xgb.DMatrix(X_test, label=y_test)

In [46]:
def objective(params):
    
    mlflow.end_run()
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(test, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [47]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[18:18:12] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:152.31629                         
[1]	validation-rmse:144.24840                         
[2]	validation-rmse:136.62095                         
[3]	validation-rmse:129.38080                         
[4]	validation-rmse:122.54012                         
[5]	validation-rmse:116.04860                         
[6]	validation-rmse:109.94951                         
[7]	validation-rmse:104.17865                         
[8]	validation-rmse:98.68816                          
[9]	validation-rmse:93.50258                          
[10]	validation-rmse:88.59894                         
[11]	validation-rmse:83.96698                         
[12]	validation-rmse:79.56124                         
[13]	validation-rmse:75.41126                         
[14]	validation-rmse:71.47752                         
[15]	validation-rmse:67.76880                         
[

In [62]:
 
params = {
    'learning_rate': 0.0502324462543933,
    'max_depth':	75,
    'min_child_weight':	19.592017909776523,
    'objective':	'reg:linear',
    'reg_alpha':	0.36230511158752154,
    'reg_lambda':	0.2713344777035102,
    'seed':	42
}

mlflow.end_run()
with mlflow.start_run():
    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(params)

    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(test, 'validation')],
        early_stopping_rounds=50)
    y_pred = booster.predict(test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[19:34:19] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:152.85409
[1]	validation-rmse:145.27360
[2]	validation-rmse:138.08283
[3]	validation-rmse:131.25744
[4]	validation-rmse:124.76020
[5]	validation-rmse:118.59694
[6]	validation-rmse:112.74079
[7]	validation-rmse:107.20838
[8]	validation-rmse:101.95065
[9]	validation-rmse:96.94473
[10]	validation-rmse:92.22210


[11]	validation-rmse:87.71454
[12]	validation-rmse:83.44227
[13]	validation-rmse:79.39034
[14]	validation-rmse:75.55353
[15]	validation-rmse:71.91189
[16]	validation-rmse:68.45362
[17]	validation-rmse:65.17905
[18]	validation-rmse:62.06477
[19]	validation-rmse:59.11363
[20]	validation-rmse:56.30478
[21]	validation-rmse:53.67993
[22]	validation-rmse:51.18163
[23]	validation-rmse:48.81268
[24]	validation-rmse:46.58306
[25]	validation-rmse:44.46794
[26]	validation-rmse:42.46373
[27]	validation-rmse:40.57166
[28]	validation-rmse:38.75952
[29]	validation-rmse:37.06063
[30]	validation-rmse:35.46487
[31]	validation-rmse:33.95620
[32]	validation-rmse:32.52733
[33]	validation-rmse:31.17437
[34]	validation-rmse:29.89189
[35]	validation-rmse:28.70932
[36]	validation-rmse:27.56121
[37]	validation-rmse:26.51694
[38]	validation-rmse:25.50111
[39]	validation-rmse:24.59427
[40]	validation-rmse:23.70653
[41]	validation-rmse:22.88766
[42]	validation-rmse:22.10094
[43]	validation-rmse:21.39021
[44]	valid

2023/07/21 19:34:25 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.


In [63]:
# 2 logging artifacts

params = {
    'learning_rate': 0.0502324462543933,
    'max_depth':	75,
    'min_child_weight':	19.592017909776523,
    'objective':	'reg:linear',
    'reg_alpha':	0.36230511158752154,
    'reg_lambda':	0.2713344777035102,
    'seed':	42
}
with mlflow.start_run():
    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(params)

    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(test, 'validation')],
        early_stopping_rounds=50)
    y_pred = booster.predict(test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(booster, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[19:48:00] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:152.85409
[1]	validation-rmse:145.27360
[2]	validation-rmse:138.08283
[3]	validation-rmse:131.25744
[4]	validation-rmse:124.76020
[5]	validation-rmse:118.59694
[6]	validation-rmse:112.74079
[7]	validation-rmse:107.20838
[8]	validation-rmse:101.95065
[9]	validation-rmse:96.94473
[10]	validation-rmse:92.22210
[11]	validation-rmse:87.71454
[12]	validation-rmse:83.44227
[13]	validation-rmse:79.39034
[14]	validation-rmse:75.55353
[15]	validation-rmse:71.91189
[16]	validation-rmse:68.45362
[17]	validation-rmse:65.17905
[18]	validation-rmse:62.06477
[19]	validation-rmse:59.11363
[20]	validation-rmse:56.30478
[21]	validation-rmse:53.67993
[22]	validation-rmse:51.18163
[23]	validation-rmse:48.81268
[24]	validation-rmse:46.58306
[25]	validation-rmse:44.46794
[26]	validation-rmse:42.46373
[27]	validation-rmse:40.57166
[28]	validation-rmse:38.75952
[29]	validat

2023/07/21 19:48:05 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
